## **Make Gradio Application**

- Run Ollama on your computer
- Create `GoogleVision` class for extract and annotate image
- Put all functions together to parse, clean, and extract information in Gradio
- Display information in Gradio

In [ ]:
# Install google cloud vision
%%capture
!pip install google-cloud
!pip install google-cloud-vision
!pip install gradio

In [ ]:
import gradio as gr
import json
import numpy as np
import pandas as pd
from google.cloud.vision import ImageAnnotatorClient, Image, EntityAnnotation
import PIL.Image as PILImage
from PIL import ImageDraw, ImageFont
from io import BytesIO
from langchain_community.llms import Ollama
from langchain import PromptTemplate # Added

FONT = ImageFont.truetype("../assets/THSarabun.ttf", 20)

class GoogleVision:
    """
    Google Vision API client

    This class allows you to recognize text in an image using Google Vision API.
    """

    def __init__(self, credential_path: str) -> None:
        """Create a Google Vision API client using the given credential path"""
        self.client: ImageAnnotatorClient = ImageAnnotatorClient.from_service_account_json(credential_path)

    def recognize(self, image: PILImage.Image) -> list[EntityAnnotation]:
        """Detect bounding box and recognize text in an image from the given PIL image"""
        # Convert PIL image to binary
        buffered = BytesIO()
        image.save(buffered, format="PNG")
        image_binary = buffered.getvalue()
        # Construct image object
        image = Image(content=image_binary)
        # Send request to Google Vision API
        response = self.client.text_detection(image)
        # Handle error.
        if response.error.message:
            raise Exception(
                f"{response.error.message}\nFor more info on error messages, check: https://cloud.google.com/apis/design/errors"
            )
        # Get all annotations except the first one (all parsed text)
        annotations = [
            annotation
            for idx, annotation in enumerate(response.text_annotations)
            if idx != 0
        ]
        return annotations

    @staticmethod
    def draw_bbox(
        image: np.array,
        annotations: list[EntityAnnotation],
        label_offset: int = 20,
    ):
        """Draw bounding box and text on the given image"""
        draw = ImageDraw.Draw(PILImage.fromarray(image))
        for annotation in annotations:
            # Get information in the annotation
            text = annotation.description
            vertices = [(vertex.x, vertex.y) for vertex in annotation.bounding_poly.vertices]
            # Draw bounding box
            draw.polygon(vertices, outline="blue")
            # Draw text
            draw.text(
                (vertices[0][0], vertices[0][1] - label_offset),
                text,
                fill="red",
                font=FONT,
            )
        return image

llm = Ollama(model="llama3.1", stop=["<|eot_id|>"]) # Added stop token
SYSTEM_PROMPT = "You are a helpful assistant expert in returning JSON output from a given prompt."

def get_model_response(user_prompt, system_prompt=SYSTEM_PROMPT):
    # NOTE: No f string and no whitespace in curly braces
    template = """
        <|begin_of_text|>
        <|start_header_id|>system<|end_header_id|>
        {system_prompt}
        <|eot_id|>
        <|start_header_id|>user<|end_header_id|>
        {user_prompt}
        <|eot_id|>
        <|start_header_id|>assistant<|end_header_id|>
        """

    # Added prompt template
    prompt = PromptTemplate(
        input_variables=["system_prompt", "user_prompt"],
        template=template
    )
    
    # Modified invoking the model
    response = llm(prompt.format(system_prompt=system_prompt, user_prompt=user_prompt))
    
    return response


def post_process(text):
    text = text.replace("วัน จดทะเบียน", "วันจดทะเบียน")
    text = text.replace("เลข ดัง", "เลขถัง")
    text = text.replace("นํ้า หนัก", "น้ำหนัก")
    text = text.replace("เลข ทะเบียน", "เลขทะเบียน")
    text = text.replace("น้ำหนัก บรรทุก", "น้ำหนักบรรทุก")
    text = text.replace("เลข ที่ บัตร", "เลขที่บัตร")
    return text

def create_prompt(text):
    prompt = f"""You are an expert in analyzing Thai vehicle registration documents. Your task is to extract specific information from the following OCR text of a Thai vehicle registration document. Please identify and extract the following information, providing the values in Thai where applicable. If a piece of information is not found or unclear, respond with "ไม่พบข้อมูล" (Information not found).
    
    OCR Text: {text}
    
    Please extract and provide the following information:
    
    1. วันจดทะเบียน (date_of_registration):
    2. เลขทะเบียน (registration_no):
    3. จังหวัด (car_province):
    4. ประเภท (vehicle_use):
    5. รย. (type):
    6. ลักษณะ (body_style):
    7. ยี่ห้อรถ (manufacturer):
    8. แบบ (model):
    9. รุ่นปี คศ (year):
    10. สี (color):
    11. เลขตัวรถ (chassis_number):
    12. อยู่ที่ (chassis_location):
    13. ยี่ห้อเครื่องยนต์ (engine_manufacturer):
    14. เลขเครื่องยนต์ (engine_number):
    15. อยู่ที่ (engine_location):
    16. เชื้อเพลิง (fuel_type):
    17. เลขถังแก๊ส or เลขดังแก๊ส (fuel_tank_number):
    18. จำนวน (cylinders):
    19. ซีซี (cubic_capacity):
    20. แรงม้า (horse_power):
    21. จำนวนเพลาและล้อ (axles_wheels_no):
    22. น้ำหนักรถ (unladen_weight):
    23. น้ำหนักบรรทุก/น้ำหนักเพลา (load_capacity):
    24. น้ำหนักรวม (gross_weight):
    25. ที่นั่ง (seats):
    
    Please provide the extracted information in a structured JSON format. Listing each item in a given key with its corresponding value. If information is not found, leave as empty string. Don't need to comment."""
    return prompt


def map_thai_to_english_keys(thai_dict):
    # Define the mapping of Thai keys to English keys
    key_mapping = {
        'วันจดทะเบียน': 'date_of_registration',
        'เลขทะเบียน': 'registration_no',
        'จังหวัด': 'car_province',
        'ประเภท': 'vehicle_use',
        'รย.': 'type',
        'ลักษณะ': 'body_style',
        'ยี่ห้อรถ': 'manufacturer',
        'แบบ': 'model',
        'รุ่นปี คศ': 'year',
        'สี': 'color',
        'เลขตัวรถ': 'chassis_number',
        'อยู่ที่': 'chassis_location',
        'ยี่ห้อเครื่องยนต์': 'engine_manufacturer',
        'เลขเครื่องยนต์': 'engine_number',
        'เชื้อเพลิง': 'fuel_type',
        'เลขถังแก๊ส': 'fuel_tank_number',
        'เลขดังแก๊ส': 'fuel_tank_number',  # Alternative key
        'จำนวน': 'cylinders',
        'ซีซี': 'cubic_capacity',
        'แรงม้า': 'horse_power',
        'จำนวนเพลาและล้อ': 'axles_wheels_no',
        'น้ำหนักรถ': 'unladen_weight',
        'น้ำหนักบรรทุก/น้ำหนักเพลา': 'load_capacity',
        'น้ำหนักรวม': 'gross_weight',
        'ที่นั่ง': 'seats'
    }
    
    # Create a new dictionary with English keys
    english_dict = {}
    for thai_key, value in thai_dict.items():
        if thai_key in key_mapping:
            english_key = key_mapping[thai_key]
            english_dict[english_key] = value
        else:
            # If the key is not in our mapping, keep the original key
            english_dict[thai_key] = value
    
    return english_dict

In [ ]:
# Create an instance of GoogleVision
PATH_TO_CREDENTIAL_JSON = "../../path_to.json"
api = GoogleVision(PATH_TO_CREDENTIAL_JSON)

image = PILImage.open("path/to/image.jpg")
annotations = api.recognize(image)  # perfrom OCR
extracted_text = " ".join([anno.description for anno in annotations])  # extract only text
extracted_text = post_process(extracted_text)  # simple post-processing

prompt = create_prompt(extracted_text)  # Create prompt for getting JSON output
output = get_model_response(prompt)  # output in text
parsed_json_output = output.split("```")[1] if "```" in output else output
parsed_json_output = json.loads(output)

df = pd.DataFrame.from_dict(parsed_json_output, orient='index', columns=['Value'])
df.reset_index(inplace=True)

## **Make Gradio Appplication**

In [ ]:
import gradio as gr

def perform_ocr(image):
    pil_image = PILImage.fromarray(image.astype('uint8'), 'RGB')
    annotations = api.recognize(pil_image)
    extracted_text = " ".join([anno.description for anno in annotations])
    extracted_text = post_process(extracted_text)
    prompt = create_prompt(extracted_text)
    output = get_model_response(prompt)
    try:
        parsed_json_output = output.split("```")[1] if "```" in output else output
        parsed_json_output = json.loads(output)
    except json.JSONDecodeError:
        return pd.DataFrame({'Error': ['Failed to parse JSON output']})
    df = pd.DataFrame.from_dict(parsed_json_output, orient='index', columns=['Value'])
    df.reset_index(inplace=True)

    drawn_image = GoogleVision.draw_bbox(image, annotations) # Drawn image
    drawn_image_array = np.array(drawn_image)
    return df, drawn_image

interface = gr.Interface(
    fn=perform_ocr,
    inputs=gr.Image(),
    outputs=[gr.Dataframe(), gr.Image()],
    title="OCR and Information Extraction",
    description="Upload an image to extract and parse information using Google Vision API and LLM."
)

# Launch the app
interface.launch()

## ฟีเจอร์ที่สามารถเพิ่มเติมได้:

ทำได้หลากหลายมาก ตัอวย่างเช่น

1. **การดาวน์โหลดผลลัพธ์**
   - เพิ่มปุ่มดาวน์โหลดไฟล์ CSV ใช้ `gr.Button`
   - และ option การเลือกรูปแบบการดาวน์โหลด (CSV, Excel, JSON) ใช้ `gradio.Radio`

2. **การปรับแต่งรูปภาพ**
   - เพิ่มเครื่องมือปรับความคมชัด
   - เพิ่มการระบบหมุนภาพอัตโนมัติ (ใช้ `deskew`)
